<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/ons_studyv7(EXP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [159]:
# @title [CELL 0] Master Template Inventory
# Store the "Stable DNA" of your 16 pages here.
# These specs act as the 'Source of Truth' for the Manifest Manager and Archiver.

PAGE_SPECS = {
    # --- Tier 1: Core Hubs & Registers ---
    "contents.shtml": {"title": "Study Guide", "type": "hub", "priority": 1, "note": "Main navigation entry point"},
    "ons_yates_dna_register.shtml": {"title": "DNA Register", "type": "data", "priority": 1, "note": "Primary lineage evidence list"},
    "ons_yates_dna_register_participants.shtml": {"title": "DNA Register (by Name)", "type": "data", "priority": 1, "note": "Participant-centric view"},

    # --- Tier 2: Visualization & Forensic Tools ---
    "dna_network.shtml": {"title": "DNA Network", "type": "visual", "priority": 2, "note": "Visual cluster analysis"},
    "lineage_proof.html": {"title": "Proof Engine", "type": "tool", "priority": 2, "note": "Interactive verification tool"},
    "dna_dossier.html": {"title": "Forensic Dossier", "type": "tool", "priority": 2, "note": "Individual evidence grading"},
    "brick_wall_buster.shtml": {"title": "Wall Buster", "type": "tool", "priority": 2, "note": "Matches unlinked lines to families"},

    # --- Tier 3: Trees & Pedigrees ---
    "just-trees.shtml": {"title": "Trees (Z-A)", "type": "pedigree", "priority": 3, "note": "Family tree list descending"},
    "just-trees-az.shtml": {"title": "Trees (A-Z)", "type": "pedigree", "priority": 3, "note": "Family tree list ascending"},

    # --- Tier 4: Utility & Engagement ---
    "subscribe.shtml": {"title": "Subscribe", "type": "engagement", "priority": 4, "note": "Groups.io integration"},
    "share_dna.shtml": {"title": "Share DNA", "type": "engagement", "priority": 4, "note": "Instructions for Ancestry sharing"},
    "gedmatch_kits.shtml": {"title": "Gedmatch Kits", "type": "data", "priority": 4, "note": "Reference for external kit IDs"},
    "print_cousin_list.shtml": {"title": "Print Cousin List", "type": "utility", "priority": 4, "note": "Printer-friendly register view"},
    "data_glossary.shtml": {"title": "Data Glossary", "type": "info", "priority": 4, "note": "Terminology definitions"},

    # --- Tier 5: Administration & Maintenance ---
    "research_admin.html": {"title": "Admin Hub", "type": "admin", "priority": 5, "note": "Study audit and metrics"},
    "site_status.html": {"title": "Deployment Manifest", "type": "audit", "priority": 5, "note": "Real-time inventory of last run"}
}

print(f"✅ Inventory Loaded: {len(PAGE_SPECS)} pages defined for tracking.")

✅ Inventory Loaded: 16 pages defined for tracking.


In [200]:
# @title [CELL 1] Setup + Helper Functions (V110 - Register Logic Restored)
import os, json, re

# --- 1. THE NAV BLOCK ---
NAV_HTML = """
<style>
nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none}
nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px;font-family:sans-serif;}
nav.oldnav a:hover{background-color:#00838f!important}
@media print { nav.oldnav, .no-print { display: none !important; } }
</style>
<nav class="oldnav"><ul>
<li><a href="research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li>
<li><a href="contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li>
<li><a href="ons_yates_dna_register.shtml">DNA Register</a></li>
<li><a href="dna_network.shtml">DNA Network</a></li>
<li><a href="lineage_proof.html">Lineage Proof</a></li>
<li><a href="dna_dossier.html">Forensic Dossier</a></li>
<li><a href="brick_wall_buster.shtml" style="background:#f57f17;color:black !important;">Brick Wall Buster</a></li>
<li><a href="data_glossary.shtml">Glossary</a></li>
<li><a href="share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li>
<li><a href="gedmatch_kits.shtml" style="background-color:#00838f; font-weight:bold;">GEDmatch Kits</a></li>
<li><a href="subscribe.shtml" style="background-color:#004d40;">Subscribe</a></li>
</ul></nav>"""

# --- 2. JAVASCRIPT CORE (Search & Sort) ---
# Extracted from V6 Cell 1 - Restores the filtering logic
JS_CORE = r"""<script type="text/javascript">
(function(){
    window.filterTable = function() {
        var input = document.getElementById("tableSearch");
        var filter = input.value.toUpperCase();
        var table = document.querySelector("table.dataframe");
        var tr = table.getElementsByTagName("tr");
        for (var i = 1; i < tr.length; i++) {
            var tdArr = tr[i].getElementsByTagName("td");
            var found = false;
            for (var j = 0; j < tdArr.length; j++) {
                if (tdArr[j]) {
                    var txtValue = tdArr[j].textContent || tdArr[j].innerText;
                    if (txtValue.toUpperCase().indexOf(filter) > -1) {
                        found = true;
                        break;
                    }
                }
            }
            tr[i].style.display = found ? "" : "none";
        }
    }
})();
</script>"""

# --- 3. ASSETS ---
GUIDE_CONTENT = """<style>.guide-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:25px;max-width:1200px;margin:30px auto;font-family:sans-serif;}.guide-card{background:white;padding:25px;border-radius:8px;border-left:5px solid #006064;box-shadow:0 4px 10px rgba(0,0,0,0.05);transition:transform 0.2s;text-align:left;}.guide-card:hover{transform:translateY(-5px)}.card-title{font-size:1.4em;font-weight:bold;color:#004d40;margin-top:0}.card-why{color:#b71c1c;font-weight:bold;margin:10px 0 5px 0;font-size:0.9em;text-transform:uppercase}.card-what{color:#555;font-size:1em;line-height:1.5;margin-bottom:20px}.card-btn{display:inline-block;padding:10px 20px;background:#00838f;color:white;text-decoration:none;border-radius:4px;font-weight:bold}.card-btn:hover{background:#006064}</style><div style="text-align:center;max-width:800px;margin:0 auto 20px auto;color:#444;font-size:1.1em;font-family:sans-serif;">This site transforms raw DNA data into forensic genealogical evidence.</div><div class="guide-grid"><div class="guide-card"><h2 class="card-title">1. The DNA Register</h2><div class="card-why">Why View This?</div><div class="card-what">To see the raw evidence. This is the master list of all matches in the study, sorted by ancestral line.</div><a href="ons_yates_dna_register.shtml" class="card-btn">Open Register</a></div><div class="guide-card"><h2 class="card-title">2. DNA Network</h2><div class="card-why">Why View This?</div><div class="card-what">To see the big picture. Visual clusters showing which ancestral lines are genetically proven by multiple testers.</div><a href="dna_network.shtml" class="card-btn">View Network</a></div><div class="guide-card"><h2 class="card-title">3. Lineage Proof Engine</h2><div class="card-why">Why View This?</div><div class="card-what">To verify a connection. An interactive tool that tests if a specific ancestor is biologically confirmed.</div><a href="lineage_proof.html" class="card-btn">Run Proof</a></div><div class="guide-card" style="border-left-color:#f57f17;"><h2 class="card-title" style="color:#e65100;">4. Brick Wall Buster</h2><div class="card-why" style="color:#bf360c;">Why View This?</div><div class="card-what">Break through a dead end. Suggests which family line you likely belong to based on match dominance.</div><a href="brick_wall_buster.shtml" class="card-btn" style="background:#ef6c00;">Bust This Wall</a></div><div class="guide-card"><h2 class="card-title">5. Forensic Dossier</h2><div class="card-why">Why View This?</div><div class="card-what">Get your "Scorecard." Generate forensic reports on yourself or an ancestor, grading evidence strength.</div><a href="dna_dossier.html" class="card-btn">Create Dossier</a></div><div class="guide-card"><h2 class="card-title">6. Research Admin Hub</h2><div class="card-why">Why View This?</div><div class="card-what">For study managers. High-level audit showing participant statistics, masked IDs, and study metrics.</div><a href="research_admin.html" class="card-btn" style="background:#455a64;">Admin Access</a></div><div class="guide-card"><h2 class="card-title">7. Data Glossary</h2><div class="card-why">Why View This?</div><div class="card-what">Understand the terms. Definitions for forensic terms like "Keystone Tester" and "Spanish naming."</div><a href="data_glossary.shtml" class="card-btn" style="background:#78909c;">Read Glossary</a></div><div class="guide-card" style="border-left-color:#fbc02d;"><h2 class="card-title">8. Corrections</h2><div class="card-why">See Something?</div><div class="card-what">Genealogy is collaboration. If you can solve a mystery, tell us. Include the <strong>Person ID (e.g. I1234)</strong>.</div><a href="mailto:ron@yates.one-name.net" class="card-btn" style="background:#f9a825;color:#333;">Email Correction</a></div></div>"""
SHARE_CONTENT = """<div style="max-width:900px; margin:0 auto; font-family:sans-serif; color:#333;"><div style="background:white; padding:40px; border-radius:8px; box-shadow:0 2px 10px rgba(0,0,0,0.1);"><h2 style="color:#006064; margin-top:0; border-bottom:2px solid #00838f; padding-bottom:10px;">Share Your Ancestry DNA Matches</h2><p style="font-size:1.1em; line-height:1.6;">Ancestry provides a built-in sharing feature...</p><h3 style="color:#00838f; margin-top:30px;">How sharing works on Ancestry</h3><p>From your AncestryDNA Settings page...</p><ul style="background:#e0f7fa; padding:15px 40px; border-radius:4px; border-left:4px solid #006064; color:#006064;"><li><strong>Viewer</strong></li><li><strong>Collaborator</strong> (Required for Study Analysis)</li><li><strong>Manager</strong></li></ul><div style="background:#f1f8e9; padding:25px; border-radius:8px; border:1px solid #c5e1a5; margin-top:30px;"><h3 style="color:#33691e; margin-top:0;">How to share for the Yates One-Name Study</h3><ol style="line-height:1.8; font-size:1.1em; color:#333;"><li>Open your <strong>AncestryDNA Settings</strong>.</li><li>Use the <strong>Invite</strong> function.</li><li>Send the invitation to <strong>Ron Yates</strong> at: <br><span style="font-family:monospace; font-weight:bold; background:white; padding:5px 10px; border:1px solid #aaa; border-radius:3px; display:inline-block; margin-top:5px; color:#c62828;">yatesvilleron@gmail.com</span></li><li>Assign the role <strong>Collaborator</strong>.</li></ol><p style="font-size:0.95em; color:#558b2f; margin-top:15px; background:white; padding:10px; border-radius:4px; border:1px solid #dcedc8;"><strong>Why Collaborator?</strong> The <em>Collaborator</em> role allows Ron to review shared matches...</p></div></div></div>"""
GEDMATCH_CONTENT = """<div style="max-width:800px; margin:0 auto; font-family:sans-serif; color:#333;"><div style="background:white; padding:30px; border-radius:8px; box-shadow:0 2px 8px rgba(0,0,0,0.1); text-align:center;"><h2 style="color:#e65100; margin-top:0;">Share Your GEDmatch Kit</h2><p style="font-size:1.1em; line-height:1.6; margin-bottom:30px;">If you have uploaded your DNA to GEDmatch, sharing your Kit Number allows us to compare it against our verified database.</p><div style="background:#fff3e0; padding:25px; border-radius:8px; border:2px dashed #ffb74d; display:inline-block; text-align:left;"><h3 style="color:#e65100; margin-top:0;">How to share:</h3><ol style="line-height:1.6; margin-bottom:20px;"><li>Log in to <strong>GEDmatch.com</strong>.</li><li>Locate your <strong>Kit Number</strong> on the dashboard (e.g., A123456).</li><li>Email the number to the study administrator.</li></ol><div style="text-align:center;"><a href="mailto:admin@yates.one-name.net?subject=GEDmatch Kit Number&body=My Kit Number is: " style="display:inline-block; padding:10px 25px; background:#ef6c00; color:white; text-decoration:none; border-radius:4px; font-weight:bold;">Email Kit Number</a></div></div></div><div style="background:white; padding:30px; border-radius:8px; box-shadow:0 2px 8px rgba(0,0,0,0.1); margin-top:30px;"><h3 style="color:#006064; margin-top:0; text-align:center; border-bottom:2px solid #00acc1; padding-bottom:10px;">Known Yates GEDmatch Kits</h3><div class="kits-table-wrap"><table class="sortable kits-table" style="width:100%; border-collapse:collapse; font-family:sans-serif;"><thead><tr><th style="background:#006064; color:white; padding:12px;">Kit Number</th><th style="background:#006064; color:white; padding:12px;">Name / Description</th></tr></thead><tbody><tr><td style="padding:10px; border-bottom:1px solid #eee;"><a href="https://www.gedmatch.com/" target="_blank" style="color:#e65100; font-weight:bold;">F201688</a></td><td style="padding:10px; border-bottom:1px solid #eee;">(Y-35 kit)</td></tr><tr><td style="padding:10px; border-bottom:1px solid #eee;"><a href="https://www.gedmatch.com/" target="_blank" style="color:#e65100; font-weight:bold;">M024169</a></td><td style="padding:10px; border-bottom:1px solid #eee;">(Y-44 kit)</td></tr></tbody></table></div></div></div>"""
SUBSCRIBE_CONTENT = """<div style="background:white; padding:40px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1); max-width:800px; margin:20px auto; text-align:center; font-family:sans-serif;"><h2 style="color:#006064; margin-top:0;">Join the Yates Research Community</h2><div style="background:#e0f2f1; padding:25px; border-radius:8px; border:1px solid #b2dfdb; display:inline-block;"><h3 style="margin-top:0; color:#004d40;">📧 One-Click Subscribe</h3><a href="mailto:yates-one-name-study+subscribe@groups.io?subject=Subscribe" style="display:inline-block; padding:15px 30px; background:#00838f; color:white; text-decoration:none; border-radius:5px; font-weight:bold;">Subscribe Now</a></div></div>"""
GLOSSARY_CONTENT = """<div style="max-width:900px; margin:0 auto; font-family:sans-serif; color:#333;"><div style="background:white; padding:40px; border-radius:8px; box-shadow:0 2px 10px rgba(0,0,0,0.1);"><h2 style="color:#006064; margin-top:0; border-bottom:2px solid #00838f; padding-bottom:10px;">Study Terminology & Methods</h2><div style="margin-bottom:30px; border-left:5px solid #0277bd; padding-left:20px;"><h3 style="color:#01579b; margin-top:0;">Forensic Handshake</h3><p style="line-height:1.6;">A robust verification method used to confirm ancestral connections. Unlike simple shared matches, a Forensic Handshake requires triangulation between three or more independent DNA kits that descend from different branches of a target ancestor. This cross-verification ensures that the genetic link is genuine and not a result of chance or endogamy.</p></div><div style="margin-bottom:30px; border-left:5px solid #ef6c00; padding-left:20px;"><h3 style="color:#e65100; margin-top:0;">Yates Bio Partners</h3><p style="line-height:1.6;">A collaborative designation for researchers who actively contribute DNA data to the study.</p><p style="line-height:1.6; background:#fff3e0; padding:15px; border-radius:4px;"><strong>Naming Convention:</strong> To accurately track lineages across generations, Bio Partners adopt the <em>Spanish Naming Method</em> for their study profiles.<br><br><strong>Format:</strong> <code>Forename Paternal-Maternal</code><br><strong>Example:</strong> <em>Robert Yates-Smith</em> indicates Robert's father was a Yates and his mother was a Smith.</p></div></div></div>"""
ADMIN_CONTENT = """<div style="max-width:1000px; margin:0 auto; font-family:sans-serif; color:#333;"><div style="text-align:center; margin-bottom:40px;"><h2 style="color:#f57f17; margin-top:0;">Researcher Control Center</h2><p style="font-size:1.1em; color:#666;">Restricted Access: Valid Researcher Credentials Required</p></div><div style="display:grid; grid-template-columns:repeat(auto-fit, minmax(300px, 1fr)); gap:30px;"><div><div style="background:white; padding:30px; border-radius:8px; box-shadow:0 4px 10px rgba(0,0,0,0.1); border-top:5px solid #f57f17;"><h3 style="color:#ef6c00; margin-top:0;">System Authentication</h3><form action="#" style="margin-top:20px;"><div style="margin-bottom:15px;"><label style="display:block; color:#666; margin-bottom:5px; font-size:0.9em;">Username</label><input type="text" style="width:100%; padding:10px; border:1px solid #ddd; border-radius:4px; box-sizing:border-box;" placeholder="researcher_id"></div><div style="margin-bottom:20px;"><label style="display:block; color:#666; margin-bottom:5px; font-size:0.9em;">Password</label><input type="password" style="width:100%; padding:10px; border:1px solid #ddd; border-radius:4px; box-sizing:border-box;" placeholder="••••••••"></div><button style="width:100%; padding:12px; background:#f57f17; color:white; border:none; border-radius:4px; font-weight:bold; cursor:pointer;">Access Dashboard</button></form><p style="text-align:center; font-size:0.85em; color:#999; margin-top:15px;">Authorized use only. All IP addresses logged.</p></div><div style="background:#263238; color:#eceff1; padding:20px; border-radius:8px; margin-top:20px;"><h4 style="margin:0 0 15px 0; color:#90a4ae; text-transform:uppercase; font-size:0.85em; letter-spacing:1px;">System Health</h4><div style="display:flex; justify-content:space-between; margin-bottom:10px; border-bottom:1px solid #37474f; padding-bottom:5px;"><span>Database Status</span><span style="color:#69f0ae;">● Online</span></div><div style="display:flex; justify-content:space-between; margin-bottom:10px; border-bottom:1px solid #37474f; padding-bottom:5px;"><span>Sync Engine</span><span style="color:#69f0ae;">● Active</span></div><div style="display:flex; justify-content:space-between;"><span>Security Protocol</span><span style="color:#40c4ff;">TLS 1.3</span></div></div></div><div><div style="background:white; padding:30px; border-radius:8px; box-shadow:0 4px 10px rgba(0,0,0,0.1); border-top:5px solid #006064;"><h3 style="color:#004d40; margin-top:0;">Administrative Tools</h3><div style="display:grid; grid-template-columns:1fr 1fr; gap:15px; margin-top:20px;"><a href="#" style="background:#e0f2f1; color:#006064; padding:20px; text-align:center; text-decoration:none; border-radius:6px; border:1px solid #b2dfdb;"><div style="font-size:1.5em; margin-bottom:5px;">📂</div><strong>Upload GEDCOM</strong></a><a href="#" style="background:#e0f2f1; color:#006064; padding:20px; text-align:center; text-decoration:none; border-radius:6px; border:1px solid #b2dfdb;"><div style="font-size:1.5em; margin-bottom:5px;">🧬</div><strong>Process Kit</strong></a><a href="#" style="background:#e0f2f1; color:#006064; padding:20px; text-align:center; text-decoration:none; border-radius:6px; border:1px solid #b2dfdb;"><div style="font-size:1.5em; margin-bottom:5px;">📝</div><strong>Edit Logs</strong></a><a href="#" style="background:#e0f2f1; color:#006064; padding:20px; text-align:center; text-decoration:none; border-radius:6px; border:1px solid #b2dfdb;"><div style="font-size:1.5em; margin-bottom:5px;">📊</div><strong>Analytics</strong></a></div><p style="font-size:0.9em; color:#777; margin-top:20px; line-height:1.5;"><em>Note: Administrative tools require Level 3 clearance. Please contact the project administrator if you need to elevate your privileges.</em></p></div></div></div></div>"""

print("✅ Cell 1: Register Features Restored (Search/Print/Toggle).")

✅ Cell 1: Register Features Restored (Search/Print/Toggle).


In [183]:
# @title [CELL 2] The Asset Manager (V114 - Local Priority)
def fetch_assets():
    print("="*60)
    print("      [CELL 2] ASSET MANAGER STARTING...")
    print("      Logic: Local Upload > Server Download")
    print("="*60)

    import os
    from ftplib import FTP_TLS
    from google.colab import userdata

    # --- CONFIGURATION ---
    SEARCH_PATHS = [
        "/tng/gedcom",           # Priority 1: User specified
        "/public_html/tng/gedcom",
        "/ons-study",
        "/public_html/ons-study",
        "/"
    ]
    KEY_FILENAME = "match_to_unmasked.csv"

    # -------------------------------------------------------
    # STEP 1: CHECK FOR MANUAL UPLOAD (PRIORITY)
    # -------------------------------------------------------
    print("[STEP 1] Checking local storage...")
    local_files = os.listdir('.')
    local_geds = [f for f in local_files if f.lower().endswith(".ged") and "unmasked" not in f.lower()]

    ged_ready = False

    if local_geds:
        # Sort by newest, just in case multiple exist
        local_geds.sort(key=lambda x: os.path.getmtime(x), reverse=True)
        print(f"    ✅ FOUND LOCAL GEDCOM: {local_geds[0]}")
        print("    --> Skipping server download. Using manual upload.")
        ged_ready = True
    else:
        print("    - No local GEDCOM found. Proceeding to server...")

    # Check for Key File
    key_ready = False
    if os.path.exists(KEY_FILENAME):
        print(f"    ✅ FOUND LOCAL KEY: {KEY_FILENAME}")
        key_ready = True

    # If we have both, we can exit early!
    if ged_ready and key_ready:
        print("\n🟢 SYSTEM READY: All assets found locally.")
        return

    # -------------------------------------------------------
    # STEP 2: DOWNLOAD FROM SERVER (FALLBACK)
    # -------------------------------------------------------
    print("\n[STEP 2] Connecting to Server (Fallback)...")
    try:
        HOST = userdata.get("FTP_HOST")
        USER = userdata.get("FTP_USER")
        PASS = userdata.get("FTP_PASS")

        ftps = FTP_TLS()
        ftps.connect(HOST, 21)
        ftps.auth()
        ftps.login(USER, PASS)
        ftps.prot_p()
        print("    ✅ Secured Connection Established.")

        # A. Download GEDCOM (If we didn't find one locally)
        if not ged_ready:
            print("    🔎 Hunting for GEDCOM on server...")
            for folder in SEARCH_PATHS:
                try:
                    ftps.cwd(folder)
                    files = ftps.nlst()
                    server_geds = [f for f in files if f.lower().endswith(".ged") and "unmasked" not in f.lower()]

                    if server_geds:
                        target = server_geds[0]
                        print(f"       Found: {target} in {folder}")
                        with open(target, "wb") as f:
                            ftps.retrbinary(f"RETR {target}", f.write)
                        print(f"       ⬇️ DOWNLOAD COMPLETE: {target}")
                        ged_ready = True
                        break
                except: continue

        # B. Download Key File (If we didn't find one locally)
        if not key_ready:
            print(f"    🔎 Hunting for {KEY_FILENAME}...")
            # Reset CWD search or assume root/study folders
            for folder in SEARCH_PATHS:
                try:
                    ftps.cwd(folder)
                    if KEY_FILENAME in ftps.nlst():
                        with open(KEY_FILENAME, "wb") as f:
                            ftps.retrbinary(f"RETR {KEY_FILENAME}", f.write)
                        print(f"       ⬇️ DOWNLOAD COMPLETE: {KEY_FILENAME}")
                        key_ready = True
                        break
                except: continue

        ftps.quit()

    except Exception as e:
        print(f"    ❌ Connection/Download Error: {e}")

    # -------------------------------------------------------
    # FINAL STATUS
    # -------------------------------------------------------
    print("-" * 60)
    if ged_ready:
        print("🟢 READY: GEDCOM is available for processing.")
    else:
        print("🔴 CRITICAL: No GEDCOM found (Locally or on Server).")
        print("   Action: Please upload a .ged file to the file browser on the left.")

fetch_assets()

      [CELL 2] ASSET MANAGER STARTING...
      Logic: Local Upload > Server Download
[STEP 1] Checking local storage...
    ✅ FOUND LOCAL GEDCOM: yates_study_2025.ged
    --> Skipping server download. Using manual upload.
    ✅ FOUND LOCAL KEY: match_to_unmasked.csv

🟢 SYSTEM READY: All assets found locally.


In [201]:
# @title [CELL 3] The Master Structure Engine (V137.3 - Restoration)
def make_page(title, content, count, view_type="", stats_bar=""):
    # Restore the exact NAV_HTML from your V6 script
    NAV_HTML = """
    <style>
    nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none}
    nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px;font-family:sans-serif;}
    nav.oldnav a:hover{background-color:#00838f!important}
    </style>
    <nav class="oldnav"><ul>
    <li><a href="research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li>
    <li><a href="contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li>
    <li><a href="ons_yates_dna_register.shtml">DNA Register</a></li>
    <li><a href="dna_network.shtml">DNA Network</a></li>
    <li><a href="lineage_proof.html">Lineage Proof</a></li>
    <li><a href="dna_dossier.html">Forensic Dossier</a></li>
    <li><a href="brick_wall_buster.shtml" style="background:#f57f17;color:black !important;">Brick Wall Buster</a></li>
    <li><a href="data_glossary.shtml">Glossary</a></li>
    <li><a href="share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li>
    <li><a href="subscribe.shtml" style="background-color:#004d40;">Subscribe</a></li>
    </ul></nav>"""

    # Restore the Methodology Banner
    SITE_INFO = """<div style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:95%;border-radius:8px;font-family:sans-serif;">
    <h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3>
    <p style="color:#333;line-height:1.6;"><strong>Methodology:</strong> blends genealogical reasoning with data-driven logic to move beyond single 'golden matches'.</p></div>"""

    html_template = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <title>{title} | Yates ONS</title>
        <style>
            body {{ font-family: sans-serif; margin:0; padding:0; background:#f5f5f5; }}
            .container {{ max-width: 1200px; margin: 0 auto; background: white; min-height: 100vh; padding-bottom: 50px; }}
            .header-banner {{ background: #006064; color: #ffcc80; padding: 20px; text-align: center; border-bottom: 4px solid #00acc1; }}
            .content-area {{ padding: 20px; }}
            table.sortable {{ width: 100%; border-collapse: collapse; margin-top: 20px; font-size: 13px; }}
            th {{ background: #00838f; color: white; padding: 10px; cursor: pointer; }}
            td {{ padding: 8px; border: 1px solid #ddd; }}
            tr:nth-child(even) {{ background: #f9f9f9; }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header-banner">
                <h1 style="margin:0;">Yates One-Name Study</h1>
                <p style="margin:5px 0 0 0;">Interactive DNA Evidence Portal</p>
            </div>
            {NAV_HTML}
            <div class="content-area">
                {stats_bar}
                {SITE_INFO if view_type != 'contents' else ''}
                <h2 style="color:#006064; border-bottom: 2px solid #00838f; padding-bottom: 5px;">{title}</h2>
                <p>Tracking <strong>{count}</strong> Verified DNA Matches</p>
                {content}
            </div>
        </div>
    </body>
    </html>
    """
    return html_template

In [204]:
# @title [CELL 4] The Publisher (V157 - Network Logic Restored)
def run_publisher():
    print("="*60)
    print("      [CELL 4] PUBLISHER STARTING (V157)")
    print("      (Status: Deploying DNA Network Clusters)")
    print("="*60)

    import os, pandas as pd, pytz
    from datetime import datetime
    from ftplib import FTP_TLS
    from google.colab import userdata

    TNG_BASE_URL = "https://yates.one-name.net/tng/verticalchart.php?personID="
    TNG_SUFFIX = "&tree=tree1&parentset=0&display=vertical&generations=15"

    def normalize_id(val):
        s = str(val).replace('@', '').strip()
        if s.isdigit(): return f"I{s}"
        return s

    # --- 1. RECONSTRUCT NARRATIVE LOGIC ---
    def build_narrative(row):
        part_name = str(row.get('Tester-Participant-Unmasked', '')).strip()
        cm = str(row.get('cM', '0'))
        anc_dated = str(row.get('Authority_FirstAncestor_dated', 'Unknown'))
        found_match = str(row.get('Found Match', 'Unknown')).strip()
        gen_count = row.get('Gen_Count', 0)
        rid = normalize_id(row.get('ID#', ''))
        linked_found_match = f'<a href="{TNG_BASE_URL}{rid}{TNG_SUFFIX}" target="_blank" style="color:#006064; text-decoration:none; font-weight:bold;">{found_match}</a>'
        return f"{part_name} is a {cm} cM DNA match to {linked_found_match} is related via {anc_dated} back {gen_count} generations."

    # --- 2. LOCAL PAGE MAKER ---
    def make_page_local(title, content, count, view_type="", stats_bar=""):
        # Register-Specific Features
        extra_ui = ""
        if view_type == 'ancestor':
            extra_ui = """
            <div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;">
                <input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;">
            </div>
            <div class="no-print" style="text-align:center;margin-bottom:15px;">
                <button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print Register</button>
            </div>
            <div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;">
                <strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="font-weight:bold;color:#006064;">By Ancestral Line</a>
            </div>"""

        SITE_INFO = """<div style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:95%;border-radius:8px;font-family:sans-serif;text-align:center;">
        <h3 style="color:#006064;margin:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3>
        <p style="color:#333;line-height:1.6;font-size:0.9em;margin-top:10px;"><strong>Methodology:</strong> blends genealogical reasoning with data-driven logic to move beyond single 'golden matches'.</p></div>"""

        methodology_box = SITE_INFO if view_type not in ['contents', 'subscribe', 'share', 'gedmatch', 'glossary', 'admin'] else ''

        return f"""<!DOCTYPE html><html lang="en">
<head><meta charset="UTF-8"><title>{title} | Yates ONS</title></head>
<body style='font-family:sans-serif;margin:0;padding:0;background:#f5f5f5;'>
    <div style='background:#006064;padding:25px;text-align:center;color:#ffcc80;border-bottom:4px solid #00acc1;'>
        <h1 style='margin:0; font-size:2.2em;'>Yates One-Name Study</h1>
        <p style='margin:8px 0 0 0; color:#e0f7fa; font-size:1.1em;'>Interactive DNA Evidence Portal</p>
    </div>
    {NAV_HTML}
    <div style='max-width:1100px;margin:20px auto;background:white;padding:30px;border-radius:8px;box-shadow: 0 4px 6px rgba(0,0,0,0.1);'>
        {stats_bar}
        {extra_ui}
        {methodology_box}
        {content}
    </div>
    {JS_CORE}
</body></html>"""

    # --- 3. DATA LOAD ---
    CSV_DB = "engine_database.csv"
    if not os.path.exists(CSV_DB): return {}
    df = pd.read_csv(CSV_DB, encoding="iso-8859-15")
    df['Long_Narrative'] = df.apply(build_narrative, axis=1)

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%B %d, %Y %-I:%M %p EST")
    stats_bar = f'<div style="background:#f4f4f4; padding:8px; text-align:center; font-size:12px; border-bottom:1px solid #ddd; margin-bottom:20px;">Update: {timestamp} | Matches: {len(df):,}</div>'

    # --- 4. GENERATE DNA NETWORK (RESTORED LOGIC) ---
    # Groups matches by Ancestor and creates expandable <details> blocks
    network_html = "<div style='font-family:sans-serif;'>"
    network_html += "<h2 style='color:#006064; border-bottom:2px solid #00acc1; padding-bottom:10px; margin-top:0;'>Genetic Cluster Network</h2>"
    network_html += "<p style='color:#666; margin-bottom:20px;'>Ancestral lines verified by multiple independent DNA testers. Click to expand a line.</p>"

    # Sort groups by size (largest clusters first)
    groups = df.groupby('Authority_FirstAncestor_dated')
    sorted_groups = sorted(groups, key=lambda x: len(x[1]), reverse=True)

    for anc, group in sorted_groups:
        count = len(group)
        # Visual badge for cluster size
        badge_color = "#f57f17" if count >= 10 else "#00838f"

        network_html += f"""
        <details style='margin-bottom:10px; border:1px solid #eee; border-radius:4px;'>
            <summary style='cursor:pointer; font-weight:bold; color:#006064; padding:10px; background:#f9f9f9;'>
                {anc}
                <span style='float:right; background:{badge_color}; color:white; padding:2px 8px; border-radius:10px; font-size:0.85em;'>{count} matches</span>
            </summary>
            <ul style='list-style-type:none; padding:15px; margin:0; border-top:1px solid #eee;'>
        """
        for _, r in group.iterrows():
            network_html += f"<li style='padding:4px 0; border-bottom:1px dashed #eee;'><strong>{r['Tester-Participant-Unmasked']}</strong> - {r['cM']} cM</li>"
        network_html += "</ul></details>"
    network_html += "</div>"

    # --- 5. UPLOAD QUEUE ---
    # Load globals safely
    try: sub_c = SUBSCRIBE_CONTENT
    except: sub_c = ""
    try: share_c = SHARE_CONTENT
    except: share_c = ""
    try: ged_c = GEDMATCH_CONTENT
    except: ged_c = ""
    try: glos_c = GLOSSARY_CONTENT
    except: glos_c = ""
    try: guide_c = GUIDE_CONTENT
    except: guide_c = ""
    try: admin_c = ADMIN_CONTENT
    except: admin_c = ""

    upload_queue = {
        "dna_network.shtml": make_page_local("DNA Network", network_html, len(df), "network", stats_bar),
        "ons_yates_dna_register.shtml": make_page_local("DNA Register", df.to_html(columns=["Long_Narrative"], index=False, classes="dataframe", escape=False, header=False), len(df), "ancestor", stats_bar),
        "contents.shtml": make_page_local("Study Guide & Tools", guide_c, len(df), "contents", stats_bar),
        "research_admin.html": make_page_local("Admin Hub", admin_c, len(df), "admin", stats_bar),
        "data_glossary.shtml": make_page_local("Study Glossary", glos_c, len(df), "glossary", stats_bar),
        "share_dna.shtml": make_page_local("Share DNA Access", share_c, len(df), "share", stats_bar),
        "gedmatch_kits.shtml": make_page_local("GEDmatch Kits", ged_c, len(df), "gedmatch", stats_bar),
        "subscribe.shtml": make_page_local("Join the Community", sub_c, len(df), "subscribe", stats_bar),
        "lineage_proof.html": make_page_local("Lineage Proof Engine", "", len(df), "proof", stats_bar),
        "brick_wall_buster.shtml": make_page_local("Brick Wall Buster", "", len(df), "buster", stats_bar),
        "dna_dossier.html": make_page_local("Forensic Dossier", "", len(df), "dossier", stats_bar),
        "just-trees.shtml": make_page_local("Trees (Z-A)", "", len(df), "tree_za", stats_bar),
        "just-trees-az.shtml": make_page_local("Trees (A-Z)", "", len(df), "tree_az", stats_bar),
        "print_cousin_list.shtml": make_page_local("Printable List", "", len(df), "print", stats_bar),
        "site_status.html": make_page_local("System Status", "", len(df), "status", stats_bar),
        "theory_of_case.shtml": make_page_local("Theory of the Case", "", len(df), "theory", stats_bar)
    }

    # --- 6. DEPLOYMENT ---
    WEB_TARGETS = ["/public_html/ons-study", "/ons-study"]
    try:
        ftps = FTP_TLS()
        ftps.connect(userdata.get("FTP_HOST"), 21)
        ftps.auth(); ftps.login(userdata.get("FTP_USER"), userdata.get("FTP_PASS")); ftps.prot_p()
        for target in WEB_TARGETS:
            try:
                ftps.cwd("/"); ftps.cwd(target)
                print(f">>> Updating {len(upload_queue)} pages in {target}...")
                for fn, content in upload_queue.items():
                    with open(fn, "w", encoding="utf-8") as f: f.write(content)
                    with open(fn, "rb") as fh: ftps.storbinary(f"STOR {fn}", fh)
            except Exception as e: print(f"   ⚠️ Skipping {target}: {e}")
        ftps.quit()
    except Exception as e: print(f"   ❌ FTP ERROR: {e}")

    return upload_queue

In [205]:
# @title [CELL 5] The Master Button (V127 - Fix Tuple Error)
def run_full_deployment():
    try:
        print(">>> ⚙️ PHASE 1: RECALCULATING DNA MATCHES...")
        run_engine()

        print("\n>>> 🚀 PHASE 2: PUBLISHING TO WEB SERVER...")
        # IMPORTANT: run_publisher returns (queue, timestamp, build_id)
        # We capture the whole thing here
        result = run_publisher()

        # Check if it returned the expected tuple or just the dictionary
        if isinstance(result, tuple):
            published_files = result[0]  # Take just the dictionary of files
        else:
            published_files = result

        if published_files:
            print(f"\n✅ WEB DEPLOYMENT SUCCESSFUL ({len(published_files)} pages)")

            print("\n>>> 📥 PHASE 3: DROPBOX SYNC & SNAPSHOT ARCHIVE...")
            # Now we are passing a clean dictionary, not a tuple
            sync_to_dropbox(published_files)

            print("\n" + "="*60)
            print(f"🎉 STUDY UPDATED SUCCESSFULLY")
            print(f"📂 Local: Check Dropbox/Apps/Yates_Study_Sync")
            print("="*60)
    except Exception as e:
        print(f"\n❌ CRITICAL SYSTEM FAILURE: {e}")

run_full_deployment()

>>> ⚙️ PHASE 1: RECALCULATING DNA MATCHES...
      [CELL 3] ENGINE STARTING (V116)
      (Feature: Bio-Identity + neeYates Logic)

[STEP 0] Checking for Required Files...
    ✅ Using GEDCOM: yates_study_2025.ged

[STEP 1] Loading Privacy Keys...
    - Loaded 94 privacy keys.
    - Generated yates_study_2025_UNMASKED.ged

[STEP 3] Tracing Lineages...
    - Parsed 63667 individuals.

[SUCCESS] Engine V116 Complete. Bio-Identities with 'neeYates' enabled.

>>> 🚀 PHASE 2: PUBLISHING TO WEB SERVER...
      [CELL 4] PUBLISHER STARTING (V157)
      (Status: Deploying DNA Network Clusters)
>>> Updating 16 pages in /public_html/ons-study...
>>> Updating 16 pages in /ons-study...

✅ WEB DEPLOYMENT SUCCESSFUL (16 pages)

>>> 📥 PHASE 3: DROPBOX SYNC & SNAPSHOT ARCHIVE...
      [CELL 9] DROPBOX SYNC & MULTI-POINT ARCHIVE
      (Target: Dropbox/Apps/Yates_Study_Sync)
>>> Mirroring 18 files for local PC access...
   ✅ MIRRORED: dna_network.shtml
   ✅ MIRRORED: ons_yates_dna_register.shtml
   ✅ MIRROR

In [ ]:
# @title [CELL 6] The Session-Capture Archiver (V2.0 - Multi-Point)
def run_archiver():
    # ... (Keep existing FTP setup) ...

    # 1. Archive the Code (Current Functionality)
    # 2. Archive the Manifest
    if os.path.exists("manifest.json"):
        with open("manifest.json", "rb") as f:
            ftps.storbinary(f"STOR manifest_{TIMESTAMP}.json", f)

    # 3. Archive HTML Snapshots (Foolproofing)
    # This saves the actual HTML strings from Cell 4's upload_queue
    snapshot_fn = f"site_snapshot_{TIMESTAMP}.json"
    with open(snapshot_fn, "w") as f:
        json.dump(published_files, f) # published_files is the upload_queue

    with open(snapshot_fn, "rb") as f:
        ftps.storbinary(f"STOR {snapshot_fn}", f)

    print(f"✅ SNAPSHOT SECURED: Site DNA versioned at {TIMESTAMP}")

In [ ]:
# @title [CELL 7] Universal Site Status Auditor
def run_site_audit():
    print("="*60)
    print("      [CELL 7] SITE STATUS AUDITOR")
    print("      Generating Live Health Checklist")
    print("="*60)

    import requests
    from ftplib import FTP_TLS
    from google.colab import userdata
    from datetime import datetime
    import pytz

    # --- 1. SETTINGS ---
    BASE_URL = "https://yates.one-name.net/ons-study/"
    est = pytz.timezone('US/Eastern')
    now = datetime.now(est).strftime("%B %d, %Y %H:%M")

    audit_results = []
    print(f"Checking {len(PAGE_SPECS)} pages...")

    # --- 2. PING THE LIVE SITE ---
    for filename, spec in PAGE_SPECS.items():
        url = f"{BASE_URL}{filename}"
        try:
            response = requests.get(url, timeout=5)
            status = "✅ LIVE" if response.status_code == 200 else f"❌ ERROR ({response.status_code})"
            color = "#2e7d32" if response.status_code == 200 else "#d32f2f"
        except:
            status = "⚠️ UNREACHABLE"
            color = "#ef6c00"

        audit_results.append({
            "name": spec['title'],
            "file": filename,
            "status": status,
            "color": color,
            "url": url
        })

    # --- 3. GENERATE AUDIT HTML ---
    rows = "".join([f"""
        <tr>
            <td style="padding:10px; border-bottom:1px solid #ddd;">{r['name']}</td>
            <td style="padding:10px; border-bottom:1px solid #ddd;"><a href="{r['url']}" target="_blank">{r['file']}</a></td>
            <td style="padding:10px; border-bottom:1px solid #ddd; font-weight:bold; color:{r['color']};">{r['status']}</td>
        </tr>
    """ for r in audit_results])

    audit_html = f"""
    <div style="font-family:sans-serif; max-width:800px; margin:20px auto; border:1px solid #ccc; border-radius:8px; padding:20px; background:white;">
        <h2 style="color:#006064; border-bottom:2px solid #006064;">Yates DNA Study: Site Health Audit</h2>
        <p><strong>Last Audit Run:</strong> {now}</p>
        <table style="width:100%; border-collapse:collapse; text-align:left;">
            <thead>
                <tr style="background:#f4f4f4;">
                    <th style="padding:10px;">Page Name</th>
                    <th style="padding:10px;">Filename</th>
                    <th style="padding:10px;">Status</th>
                </tr>
            </thead>
            <tbody>{rows}</tbody>
        </table>
    </div>
    """

    # --- 4. UPLOAD AUDIT TO SERVER ---
    try:
        ftps = FTP_TLS()
        ftps.connect(userdata.get("FTP_HOST"), 21)
        ftps.auth(); ftps.login(userdata.get("FTP_USER"), userdata.get("FTP_PASS"))
        ftps.prot_p()

        with open("site_status.html", "w") as f: f.write(audit_html)
        with open("site_status.html", "rb") as f:
            ftps.storbinary("STOR site_status.html", f)

        ftps.quit()
        print(f"\n✨ AUDIT COMPLETE: View at {BASE_URL}site_status.html")
    except Exception as e:
        print(f"❌ Audit Upload Failed: {e}")

run_site_audit()

      [CELL 7] SITE STATUS AUDITOR
      Generating Live Health Checklist
Checking 16 pages...

✨ AUDIT COMPLETE: View at https://yates.one-name.net/ons-study/site_status.html


In [ ]:
# @title [CELL 8] Deployment Location Tracker (GPS + Live Inventory)
def track_deployment_path():
    print("="*60)
    print("      [CELL 8] DEPLOYMENT TRACKER (GPS)")
    print("      Mapping Server & Listing Live Pages")
    print("="*60)

    from ftplib import FTP_TLS
    from google.colab import userdata

    # Priority paths for Yates Study on Guild Server
    potential_roots = ["/public_html/yates", "/public_html", "/members_websites/yates", "/"]
    BASE_URL = "https://yates.one-name.net/ons-study/"
    found_live_files = []
    active_path = None

    try:
        ftps = FTP_TLS()
        ftps.connect(userdata.get("FTP_HOST"), 21)
        ftps.auth(); ftps.login(userdata.get("FTP_USER"), userdata.get("FTP_PASS"))
        ftps.prot_p()

        # STEP 1: Find the Active Path
        for root in potential_roots:
            try:
                ftps.cwd("/")
                ftps.cwd(root)
                items = [i.lower() for i in ftps.nlst()]
                if "ons-study" in items:
                    active_path = f"{root}/ons-study"
                    ftps.cwd("ons-study")
                    found_live_files = ftps.nlst()
                    break
            except: continue

        # STEP 2: Display Results with Hotlinks
        print("\n" + "="*60)
        if active_path:
            print(f"✅ ACTIVE WEB ROOT FOUND: {active_path}")
            print(f"📊 LIVE INVENTORY ({len(found_live_files)} files found):")
            print("-" * 60)

            # Sort files to match your 16-page inventory order
            found_live_files.sort()

            for filename in found_live_files:
                if filename.endswith(('.shtml', '.html', '.php', '.csv')):
                    # Generate the direct clickable hotlink
                    hotlink = f"{BASE_URL}{filename}"
                    print(f"🔗 {filename.ljust(35)} -> {hotlink}")

            print("-" * 60)
            print(f"📍 COPY THIS PATH TO CELL 4: TARGET_PATH = '{active_path}'")
        else:
            print("❌ NO STUDY FOLDER DETECTED. Check if files were uploaded to a private root.")

        ftps.quit()
        print("="*60)

    except Exception as e:
        print(f"❌ Connection Error: {e}")

track_deployment_path()

      [CELL 8] DEPLOYMENT TRACKER (GPS)
      Mapping Server & Listing Live Pages

✅ ACTIVE WEB ROOT FOUND: /public_html/yates/ons-study
📊 LIVE INVENTORY (18 files found):
------------------------------------------------------------
🔗 brick_wall_buster.shtml             -> https://yates.one-name.net/ons-study/brick_wall_buster.shtml
🔗 contents.shtml                      -> https://yates.one-name.net/ons-study/contents.shtml
🔗 data_glossary.shtml                 -> https://yates.one-name.net/ons-study/data_glossary.shtml
🔗 dna_dossier.html                    -> https://yates.one-name.net/ons-study/dna_dossier.html
🔗 dna_network.shtml                   -> https://yates.one-name.net/ons-study/dna_network.shtml
🔗 gedmatch_kits.shtml                 -> https://yates.one-name.net/ons-study/gedmatch_kits.shtml
🔗 just-trees-az.shtml                 -> https://yates.one-name.net/ons-study/just-trees-az.shtml
🔗 just-trees.shtml                    -> https://yates.one-name.net/ons-study/just-tree

In [63]:
# @title [CELL 8.5] Zero-Touch Token Exchange
import requests
from google.colab import userdata

# This pulls directly from your 🔑 Secrets to avoid placeholder errors
try:
    AK = userdata.get('DBX_APP_KEY')
    AS = userdata.get('DBX_APP_SECRET')

    if AK == "your_actual_key_here" or AK is None:
        print("❌ STOP: You need to go to the 🔑 Secrets tab and set 'DBX_APP_KEY'.")
    else:
        # STEP 1: Generate the link
        auth_url = f"https://www.dropbox.com/oauth2/authorize?client_id={AK}&token_access_type=offline&response_type=code"
        print(f"✅ STEP 1: Click this link to authorize your app:\n{auth_url}\n")

        # STEP 2: Input the result
        auth_code = input("✅ STEP 2: Paste the 'Authorization Code' from the browser here: ").strip()

        if auth_code:
            url = "https://api.dropbox.com/oauth2/token"
            data = {
                "code": auth_code,
                "grant_type": "authorization_code",
                "client_id": AK,
                "client_secret": AS,
            }
            res = requests.post(url, data=data).json()

            if "refresh_token" in res:
                print("\n" + "="*60)
                print(f"🚀 FINAL REFRESH TOKEN: {res['refresh_token']}")
                print("="*60)
                print("👉 ACTION: Save this in 🔑 Secrets as 'DBX_REFRESH_TOKEN'")
            else:
                print("\n❌ ERROR FROM DROPBOX:", res)
except Exception as e:
    print(f"❌ SECRET ERROR: Ensure you have DBX_APP_KEY and DBX_APP_SECRET in your secrets. {e}")

✅ STEP 1: Click this link to authorize your app:
https://www.dropbox.com/oauth2/authorize?client_id=q5oeqa22wn0ls9i&token_access_type=offline&response_type=code

✅ STEP 2: Paste the 'Authorization Code' from the browser here: SJIDDoAMoCEAAAAAAAMzvdI4kw6t8XIcBnTS7t0TcrA

🚀 FINAL REFRESH TOKEN: 32bAqhurRt4AAAAAAAAAATIEks0IEjX8I0VheJGpU6ptEysjM1rbRRgU5Ci5MEfo
👉 ACTION: Save this in 🔑 Secrets as 'DBX_REFRESH_TOKEN'


In [93]:
# @title [CELL 9] Dropbox Sync & Multi-Point Archiver (V2.1)
import dropbox, os, json, time
from google.colab import userdata

def sync_to_dropbox(published_files=None):
    """
    Accepts 'published_files' from Cell 5 to create
    the versioned Site Snapshots you requested.
    """
    print("="*60)
    print("      [CELL 9] DROPBOX SYNC & MULTI-POINT ARCHIVE")
    print("      (Target: Dropbox/Apps/Yates_Study_Sync)")
    print("="*60)

    try:
        # 1. CONNECT
        dbx = dropbox.Dropbox(
            app_key=userdata.get('DBX_APP_KEY'),
            app_secret=userdata.get('DBX_APP_SECRET'),
            oauth2_refresh_token=userdata.get('DBX_REFRESH_TOKEN')
        )

        TIMESTAMP = time.strftime("%Y-%m-%d_%H%M_%S")
        ARCHIVE_DIR = "/archives"

        # Ensure archive directory exists (Dropbox creates it on upload if path is full)

        # ---------------------------------------------------------
        # PART A: THE LIVE MIRROR (1:1 Files for Local Testing)
        # ---------------------------------------------------------
        # We sync everything from the publisher + the database
        files_to_mirror = list(published_files.keys()) if published_files else []
        files_to_mirror.extend(["engine_database.csv", "manifest.json"])

        print(f">>> Mirroring {len(files_to_mirror)} files for local PC access...")
        for f_name in files_to_mirror:
            if os.path.exists(f_name):
                with open(f_name, "rb") as f:
                    dbx.files_upload(f.read(), f"/{f_name}", mode=dropbox.files.WriteMode.overwrite)
                print(f"   ✅ MIRRORED: {f_name}")

        # ---------------------------------------------------------
        # PART B: MULTI-POINT ARCHIVE (The 'Foolproofing' Layer)
        # ---------------------------------------------------------
        print("\n>>> SECURING SNAPSHOTS...")

        # 1. Archive the Code (Current Notebook Logic)
        from IPython import get_ipython
        hm = get_ipython().history_manager
        code_content = "\n\n".join([f"# --- CELL ---\n{cell[2]}" for cell in hm.get_range()])
        dbx.files_upload(code_content.encode('utf-8'), f"{ARCHIVE_DIR}/code_capture_{TIMESTAMP}.py")
        print(f"   ✅ ARCHIVED: code_capture_{TIMESTAMP}.py")

        # 2. Archive the Site Snapshot (The HTML strings you requested)
        if published_files:
            snapshot_fn = f"site_snapshot_{TIMESTAMP}.json"
            snapshot_bytes = json.dumps(published_files).encode('utf-8')
            dbx.files_upload(snapshot_bytes, f"{ARCHIVE_DIR}/{snapshot_fn}")
            print(f"   ✅ SNAPSHOT SECURED: {snapshot_fn}")

        print("\n" + "="*60)
        print(f"✨ SUCCESS: Sync and Archive complete at {TIMESTAMP}")
        print("="*60)

    except Exception as e:
        print(f"❌ DROPBOX ERROR: {e}")

# Cell is now defined and ready to receive data from Cell 5.